<a href="https://colab.research.google.com/github/hedi11999/crop_classification_scotland_using_satelitte_imagery_major_tom/blob/main/data_collection_annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio
!pip install h5py
!pip install folium geopandas



# Major TOM Code Integration

The following code has been sourced from the Major TOM project on GitHub. This code is designed for handling and processing geospatial data, including reading, filtering, and visualizing satellite imagery. We have combined several modules from the Major TOM repository into a single notebook to streamline the data collection and processing workflow. This integration allows us to easily work with the dataset in a unified environment.


In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from pathlib import Path
import rasterio as rio
from PIL import Image
import torchvision.transforms as transforms

class MajorTOM(Dataset):
    """MajorTOM Dataset (https://huggingface.co/Major-TOM)

    Args:
        df ((geo)pandas.DataFrame): Metadata dataframe
        local_dir (string): Root directory of the local dataset version
        tif_bands (list): A list of tif file names to be read
        png_bands (list): A list of png file names to be read

    """

    def __init__(self,
                 df,
                 local_dir = None,
                 tif_bands=['B04','B03','B02'],
                 png_bands=['thumbnail'],
                 tif_transforms=[transforms.ToTensor()],
                 png_transforms=[transforms.ToTensor()]
                ):
        super().__init__()
        self.df = df
        self.local_dir = Path(local_dir) if isinstance(local_dir,str) else local_dir
        self.tif_bands = tif_bands if not isinstance(tif_bands,str) else [tif_bands]
        self.png_bands = png_bands if not isinstance(png_bands,str) else [png_bands]
        self.tif_transforms = transforms.Compose(tif_transforms) if tif_transforms is not None else None
        self.png_transforms = transforms.Compose(png_transforms) if png_transforms is not None else None

    def __len__(self):
        return len(self.df)

    def print_columns(self):
        print("Columns:")
        for column in self.df.columns:
            print(column)

    def __getitem__(self, idx):
        meta = self.df.iloc[idx]

        product_id = meta.product_id
        grid_cell = meta.grid_cell
        row = grid_cell.split('_')[0]

        path = self.local_dir / Path("{}/{}/{}".format(row, grid_cell, product_id))
        out_dict = {'meta' : meta}

        for band in self.tif_bands:
            with rio.open(path / '{}.tif'.format(band)) as f:
                out = f.read()
            if self.tif_transforms is not None:
                out = self.tif_transforms(out)
            out_dict[band] = out


        for band in self.png_bands:
            out = Image.open(path / '{}.png'.format(band))
            if self.png_transforms is not None:
                out = self.png_transforms(out)
            out_dict[band] = out

        return out_dict

import numpy as np
import math
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Polygon
from tqdm import tqdm



class Grid():

    RADIUS_EQUATOR = 6378.137 # km

    def __init__(self,dist,latitude_range=(-85,85),longitude_range=(-180,180),utm_definition='bottomleft'):
        self.dist = dist
        self.latitude_range = latitude_range
        self.longitude_range = longitude_range
        self.utm_definition = utm_definition
        self.rows,self.lats = self.get_rows()
        self.points, self.points_by_row = self.get_points()

    def get_rows(self):

        # Define set of latitudes to use, based on the grid distance
        arc_pole_to_pole = math.pi * self.RADIUS_EQUATOR
        num_divisions_in_hemisphere = math.ceil(arc_pole_to_pole / self.dist)

        latitudes = np.linspace(-90, 90, num_divisions_in_hemisphere+1)[:-1]
        latitudes = np.mod(latitudes, 180) - 90

        # order should be from south to north
        latitudes = np.sort(latitudes)

        zeroth_row = np.searchsorted(latitudes,0)

        # From 0U-NU and 1D-ND
        rows = [None] * len(latitudes)
        rows[zeroth_row:] = [f'{i}U' for i in range(len(latitudes)-zeroth_row)]
        rows[:zeroth_row] = [f'{abs(i-zeroth_row)}D' for i in range(zeroth_row)]

        # bound to range
        idxs = (latitudes>=self.latitude_range[0]) * (latitudes<=self.latitude_range[1])
        rows,latitudes = np.array(rows), np.array(latitudes)
        rows,latitudes = rows[idxs],latitudes[idxs]

        return rows,latitudes

    def get_circumference_at_latitude(self,lat):

        # Circumference of the cross-section of a sphere at a given latitude

        radius_at_lat = self.RADIUS_EQUATOR * math.cos(lat * math.pi / 180)
        circumference = 2 * math.pi * radius_at_lat

        return circumference

    def subdivide_circumference(self,lat,return_cols=False):
        # Provide a list of longitudes that subdivide the circumference of the earth at a given latitude
        # into equal parts as close as possible to dist

        circumference = self.get_circumference_at_latitude(lat)
        num_divisions = math.ceil(circumference / self.dist)
        longitudes = np.linspace(-180,180, num_divisions+1)[:-1]
        longitudes = np.mod(longitudes, 360) - 180
        longitudes = np.sort(longitudes)


        if return_cols:
            cols = [None] * len(longitudes)
            zeroth_idx = np.where(longitudes==0)[0][0]
            cols[zeroth_idx:] = [f'{i}R' for i in range(len(longitudes)-zeroth_idx)]
            cols[:zeroth_idx] = [f'{abs(i-zeroth_idx)}L' for i in range(zeroth_idx)]
            return np.array(cols),np.array(longitudes)

        return np.array(longitudes)

    def get_points(self):

        r_idx = 0
        points_by_row = [None]*len(self.rows)
        for r,lat in zip(self.rows,self.lats):
            point_names,grid_row_names,grid_col_names,grid_row_idx,grid_col_idx,grid_lats,grid_lons,utm_zones,epsgs = [],[],[],[],[],[],[],[],[]
            cols,lons = self.subdivide_circumference(lat,return_cols=True)

            cols,lons = self.filter_longitude(cols,lons)
            c_idx = 0
            for c,lon in zip(cols,lons):
                point_names.append(f'{r}_{c}')
                grid_row_names.append(r)
                grid_col_names.append(c)
                grid_row_idx.append(r_idx)
                grid_col_idx.append(c_idx)
                grid_lats.append(lat)
                grid_lons.append(lon)
                if self.utm_definition == 'bottomleft':
                    utm_zones.append(get_utm_zone_from_latlng([lat,lon]))
                elif self.utm_definition == 'center':
                    center_lat = lat + (1000*self.dist/2)/111_120
                    center_lon = lon + (1000*self.dist/2)/(111_120*math.cos(center_lat*math.pi/180))
                    utm_zones.append(get_utm_zone_from_latlng([center_lat,center_lon]))
                else:
                    raise ValueError(f'Invalid utm_definition {self.utm_definition}')
                epsgs.append(f'EPSG:{utm_zones[-1]}')

                c_idx += 1
            points_by_row[r_idx] = gpd.GeoDataFrame({
                'name':point_names,
                'row':grid_row_names,
                'col':grid_col_names,
                'row_idx':grid_row_idx,
                'col_idx':grid_col_idx,
                'utm_zone':utm_zones,
                'epsg':epsgs
            },geometry=gpd.points_from_xy(grid_lons,grid_lats))
            r_idx += 1
        points = gpd.GeoDataFrame(pd.concat(points_by_row))
        # points.reset_index(inplace=True,drop=True)
        return points, points_by_row

    def group_points_by_row(self):
        # Make list of different gdfs for each row
        points_by_row = [None]*len(self.rows)
        for i,row in enumerate(self.rows):
            points_by_row[i] = self.points[self.points.row==row]
        return points_by_row

    def filter_longitude(self,cols,lons):
        idxs = (lons>=self.longitude_range[0]) * (lons<=self.longitude_range[1])
        cols,lons = cols[idxs],lons[idxs]
        return cols,lons

    def latlon2rowcol(self,lats,lons,return_idx=False,integer=False):
        """
        Convert latitude and longitude to row and column number from the grid
        """
        # Always take bottom left corner of grid cell
        rows = np.searchsorted(self.lats,lats)-1

        # Get the possible points of the grid cells at the given latitude
        possible_points = [self.points_by_row[row] for row in rows]

        # For each point, find the rightmost point that is still to the left of the given longitude
        cols = [poss_points.iloc[np.searchsorted(poss_points.geometry.x,lon)-1].col for poss_points,lon in zip(possible_points,lons)]
        rows = self.rows[rows].tolist()

        outputs = [rows, cols]
        if return_idx:
            # Get the table index for self.points with each row,col pair in rows, cols
            idx = [self.points[(self.points.row==row) & (self.points.col==col)].index.values[0] for row,col in zip(rows,cols)]
            outputs.append(idx)

        # return raw numbers
        if integer:
            outputs[0] = [int(el[:-1]) if el[-1] == 'U' else -int(el[:-1]) for el in outputs[0]]
            outputs[1] = [int(el[:-1]) if el[-1] == 'R' else -int(el[:-1]) for el in outputs[1]]

        return outputs

    def rowcol2latlon(self,rows,cols):
        point_geoms = [self.points.loc[(self.points.row==row) & (self.points.col==col),'geometry'].values[0] for row,col in zip(rows,cols)]
        lats = [point.y for point in point_geoms]
        lons = [point.x for point in point_geoms]
        return lats,lons

    def get_bounded_footprint(self,point,buffer_ratio=0):
        # Gets the polygon footprint of the grid cell for a given point, bounded by the other grid points' cells.
        # Grid point defined as bottom-left corner of polygon. Buffer ratio is the ratio of the grid cell's width/height to buffer by.

        bottom,left = point.geometry.y,point.geometry.x
        row = point.row
        row_idx = point.row_idx
        col_idx = point.col_idx
        next_row_idx = row_idx+1
        next_col_idx = col_idx+1

        if next_row_idx >= len(self.lats): # If at top row, use difference between top and second-to-top row for height
            height = (self.lats[row_idx] - self.lats[row_idx-1])
            top = self.lats[row_idx] + height
        else:
            top = self.lats[next_row_idx]

        max_col = len(self.points_by_row[row].col_idx)-1
        if next_col_idx > max_col: # If at rightmost column, use difference between rightmost and second-to-rightmost column for width
            width = (self.points_by_row[row].iloc[col_idx].geometry.x - self.points_by_row[row].iloc[col_idx-1].geometry.x)
            right = self.points_by_row[row].iloc[col_idx].geometry.x + width
        else:
            right = self.points_by_row[row].iloc[next_col_idx].geometry.x

        # Buffer the polygon by the ratio of the grid cell's width/height
        width = right - left
        height = top - bottom

        buffer_horizontal = width * buffer_ratio
        buffer_vertical = height * buffer_ratio

        new_left = left - buffer_horizontal
        new_right = right + buffer_horizontal

        new_bottom = bottom - buffer_vertical
        new_top = top + buffer_vertical

        bbox = Polygon([(new_left,new_bottom),(new_left,new_top),(new_right,new_top),(new_right,new_bottom)])

        return bbox


def get_utm_zone_from_latlng(latlng):
    """
    Get the UTM ZONE from a latlng list.

    Parameters
    ----------
    latlng : List[Union[int, float]]
        The latlng list to get the UTM ZONE from.

    return_epsg : bool, optional
        Whether or not to return the EPSG code instead of the WKT, by default False

    Returns
    -------
    str
        The WKT or EPSG code.
    """
    assert isinstance(latlng, (list, np.ndarray)), "latlng must be in the form of a list."

    zone = math.floor(((latlng[1] + 180) / 6) + 1)
    n_or_s = "S" if latlng[0] < 0 else "N"

    false_northing = "10000000" if n_or_s == "S" else "0"
    central_meridian = str(zone * 6 - 183)
    epsg = f"32{'7' if n_or_s == 'S' else '6'}{str(zone)}"

    return epsg


if __name__ == '__main__':
    import matplotlib.pyplot as plt

    dist = 100
    grid = Grid(dist,latitude_range=(10,70),longitude_range=(-30,60))

    from pprint import pprint

    test_lons = np.random.uniform(-20,50,size=(1000))
    test_lats = np.random.uniform(12,68,size=(1000))

    test_rows,test_cols = grid.latlon2rowcol(test_lats,test_lons)
    test_lats2,test_lons2 = grid.rowcol2latlon(test_rows,test_cols)

    print(test_lons[:10])
    print(test_lats[:10])
    print(test_rows[:10])
    print(test_cols[:10])

    # Make line segments from the points to their corresponding grid points
    lines = []
    for i in range(len(test_lats)):
        lines.append([(test_lons[i],test_lats[i]),(test_lons2[i],test_lats2[i])])

    lines = gpd.GeoDataFrame(geometry=gpd.GeoSeries([LineString(line) for line in lines]))

    lines.to_file(f'testlines_{dist}km.geojson',driver='GeoJSON')
    grid.points.to_file(f'testgrid_{dist}km.geojson',driver='GeoJSON')

import pyarrow.parquet as pq
import pandas as pd
import geopandas as gpd
from pathlib import Path
import urllib.request
import fsspec
from fsspec.parquet import open_parquet_file
from io import BytesIO
from PIL import Image
from rasterio.io import MemoryFile
from tqdm.notebook import tqdm
import os

def metadata_from_url(access_url, local_url):
    local_url, response = urllib.request.urlretrieve(access_url, local_url)
    df = pq.read_table(local_url).to_pandas()
    df['timestamp'] = pd.to_datetime(df.timestamp)
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.centre_lon, df.centre_lat), crs=df.crs.iloc[0]
    )
    return gdf

def filter_metadata(df,
                    region=None,
                    daterange=None,
                    cloud_cover=(0,100),
                    nodata=(0, 1.0)
                   ):
    """Filters the Major-TOM dataframe based on several parameters

    Args:
        df (geopandas dataframe): Parent dataframe
        region (shapely geometry object) : Region of interest
        daterange (tuple) : Inclusive range of dates (example format: '2020-01-01')
        cloud_cover (tuple) : Inclusive percentage range (0-100) of cloud cover
        nodata (tuple) : Inclusive fraction (0.0-1.0) of no data allowed in a sample

    Returns:
        df: a filtered dataframe
    """
    # temporal filtering
    if daterange is not None:
        assert (isinstance(daterange, list) or isinstance(daterange, tuple)) and len(daterange)==2
        df = df[df.timestamp >= daterange[0]]
        df = df[df.timestamp <= daterange[1]]

    # spatial filtering
    if region is not None:
        idxs = df.sindex.query(region)
        df = df.take(idxs)
    # cloud filtering
    if cloud_cover is not None:
        df = df[df.cloud_cover >= cloud_cover[0]]
        df = df[df.cloud_cover <= cloud_cover[1]]

    # spatial filtering
    if nodata is not None:
        df = df[df.nodata >= nodata[0]]
        df = df[df.nodata <= nodata[1]]

    return df

def read_row(row, columns=["thumbnail"]):
    """Reads a row from a Major-TOM dataframe

    Args:
        row (row from geopandas dataframe): The row of metadata
        columns (list): columns to be read from the file

    Returns:
        data (dict): dictionary with returned data from requested columns
    """
    with open_parquet_file(row.parquet_url,columns = columns) as f:
        with pq.ParquetFile(f) as pf:
            row_group = pf.read_row_group(row.parquet_row, columns=columns)

    if columns == ["thumbnail"]:
        stream = BytesIO(row_group['thumbnail'][0].as_py())
        return Image.open(stream)
    else:
        row_output = {}
        for col in columns:
            bytes = row_group[col][0].as_py()

            if col != 'thumbnail':
                row_output[col] = read_tif_bytes(bytes)
            else:
                stream = BytesIO(bytes)
                row_output[col] = Image.open(stream)

        return row_output

def filter_download(df, local_dir, source_name, by_row = False, verbose = False, tif_columns=None):
    """Downloads and unpacks the data of Major-TOM based on a metadata dataframe

    Args:
        df (geopandas dataframe): Metadata dataframe
        local_dir (str or Path) : Path to the where the data is to be stored locally
        source_name (str) : Name alias of the resulting dataset
        by_row (bool): If True, it will access individual rows of parquet via http - otherwise entire parquets are downloaded temporarily
        verbose (bool) : option for potential internal state printing

    Returns:
        None

    """

    if isinstance(local_dir, str):
        local_dir = Path(local_dir)

    temp_file = local_dir / 'temp.parquet'

    # identify all parquets that need to be downloaded (group them)
    urls = df.parquet_url.unique()
    print('Starting download of {} parquet files.'.format(len(urls))) if verbose else None

    for url in tqdm(urls, desc='Downloading and unpacking...'):
        # identify all relevant rows
        rows = df[df.parquet_url == url].parquet_row.unique()

        if not by_row: # (downloads entire parquet)
            # download a temporary file
            temp_path, http_resp = urllib.request.urlretrieve(url, temp_file)
        else:
            f=fsspec.open(url)
            temp_path = f.open()

        # populate the bands
        with pq.ParquetFile(temp_path) as pf:
            for row_idx in rows:
                table = pf.read_row_group(row_idx)

                product_id = table['product_id'][0].as_py()
                grid_cell = table['grid_cell'][0].as_py()
                row = grid_cell.split('_')[0]

                dest = local_dir / Path("{}/{}/{}/{}".format(source_name, row, grid_cell, product_id))
                dest.mkdir(exist_ok=True, parents=True)

                columns = [col for col in table.column_names if col[0] == 'B'] + ['cloud_mask'] if tif_columns is None else tif_columns
                # tifs
                for col in columns:
                    with open(dest / "{}.tif".format(col), "wb") as f:
                        # Write bytes to file
                        f.write(table[col][0].as_py())

                # thumbnail (png)
                col = 'thumbnail'
                with open(dest / "{}.png".format(col), "wb") as f:
                    # Write bytes to file
                    f.write(table[col][0].as_py())
        if not by_row:
            # remove downloaded file
            os.remove(temp_path)
        else:
            f.close()

from rasterio.io import MemoryFile
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

def plot(sample, bands = ['B04', 'B03', 'B02'], scaling=2e3):
    img = []
    for b in bands:
        img.append(read_tif_bytes(sample[b]))
    plt.imshow(np.stack(img, -1)/2e3)

def read_tif_bytes(tif_bytes):
    with MemoryFile(tif_bytes) as mem_f:
        with mem_f.open(driver='GTiff') as f:
            return f.read().squeeze()

def read_png_bytes(png_bytes):
    stream = BytesIO(png_bytes)
    return Image.open(stream)



# Data Filtering and Download with Major TOM

In this section, we utilize the Major TOM code from GitHub to filter and download Sentinel-2 satellite imagery data based on specific temporal and spatial parameters. The filtering process is configured to:

1. **Temporal Range**: We filter the data for the year 2018, from January 1st, 2018, to January 1st, 2019.
2. **Cloud Coverage**: We only include images with cloud coverage between 0% and 20%.
3. **Region**: The geographical focus is on East Scotland, defined by a bounding box around agricultural regions near Dundee and Edinburgh.

While this example focuses on Sentinel-2 data, it may be beneficial to also consider data from other satellites, such as combining it with Sentinel-1 SAR data. Exploring the use of both optical and radar imagery could provide a more comprehensive analysis.



In [ ]:
from pathlib import Path
import urllib.request

SOURCE_DATASET = 'Major-TOM/Core-S2L2A' # Identify HF Dataset
DATASET_DIR = Path('./data/Major-TOM/')
DATASET_DIR.mkdir(exist_ok=True, parents=True)
ACCESS_URL = 'https://huggingface.co/datasets/{}/resolve/main/metadata.parquet?download=true'.format(SOURCE_DATASET)
LOCAL_URL = DATASET_DIR / '{}.parquet'.format(ACCESS_URL.split('.parquet')[0].split('/')[-1])

# download from server to local url
gdf = metadata_from_url(ACCESS_URL, LOCAL_URL)

print(gdf.head())

from shapely.geometry import box

# Example bounding boxes used for filtering
switzerland = box(5.9559111595,45.8179931641,10.4920501709,47.808380127)
gabon = box(8.1283659854,-4.9213919841,15.1618722208,2.7923006325)
napoli = box(14.091710578,40.7915558593,14.3723765416,40.9819258062)
pacific = box(-153.3922893485,39.6170415622,-152.0423077748,40.7090892316) # a remote patch over pacific - no data
scotland = box(-4.5, 55.0, -2.0, 57.0) # east Scotland agricultural regions arround dundee and arround edinburgh



filtered_df = filter_metadata(gdf,
                              cloud_cover = (0,20), # cloud cover between 0% and 20%
                              region=scotland, # you can try with different bounding boxes, like in the cell above
                              daterange=('2018-01-01', '2019-01-01'), # temporal range
                              nodata=(0.0,0.0) # only 0% of no data allowed
                              )

print(filtered_df.head())

filter_download(filtered_df, local_dir='./data/', source_name='L2A', by_row=True)

# Data Annotation and Exploration

With the Sentinel-2 data successfully downloaded, the next step involves exploring the dataset to understand its content and prepare for annotation. This section will begin by visualizing some of the images to get a sense of the data quality and coverage. We will also check the number of images available to us, which is crucial for planning the annotation process.


In [ ]:
ds = MajorTOM(filtered_df, './data/L2A')

ds[0]

print(len(ds))

print(type(ds))

# Print the type and some example data to understand the structure of 'ds'
print("Type of ds:", type(ds))
print("Type of first element in ds:", type(ds[0]))
print("First element in ds:", ds[0])






# Combining Crop Maps into a Single GeoDataFrame

In this step, we will combine all crop map files from different regions into a single GeoDataFrame. This unified GeoDataFrame will contain all the polygons representing the crop maps for the specific region under study. By consolidating the data into one GeoDataFrame, we can streamline further analysis and processing tasks, ensuring that all relevant spatial information is easily accessible and manageable.


In [ ]:
import os
import glob
import geopandas as gpd
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Path to your GPKG folder in Google Drive
gpkg_folder_path = '/content/drive/My Drive/MscProject/CropMaps/2023/'

# List all GPKG files in the folder
gpkg_files = glob.glob(os.path.join(gpkg_folder_path, '*.gpkg'))

# Initialize an empty list to hold GeoDataFrames
gdfs = []

# Read each GPKG file and append the GeoDataFrame to the list
for gpkg_file in gpkg_files:
    gdf = gpd.read_file(gpkg_file)
    gdfs.append(gdf)

# Concatenate all GeoDataFrames into one
combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

# Print the combined GeoDataFrame
print(combined_gdf.head())

# Check the CRS of the combined GeoDataFrame
print(combined_gdf.crs)

# If needed, transform the combined GeoDataFrame to the target CRS (for example, EPSG:4326 - WGS 84)
target_crs = "epsg:4326"
combined_gdf_transformed = combined_gdf.to_crs(target_crs)

# Display the transformed GeoDataFrame
print(combined_gdf_transformed.head())

# Print unique values in the 'crop_code' column
unique_crop_codes = combined_gdf_transformed['crop_code'].unique()
print("Unique Crop Codes:", unique_crop_codes)

# Print unique values in the 'crop_name' column
unique_crop_names = combined_gdf_transformed['crop_name'].unique()
print("Unique Crop Names:", unique_crop_names)

# Example: Access the first polygon's coordinates
if not combined_gdf_transformed.empty:
    first_polygon = combined_gdf_transformed.loc[0, 'geometry']
    if first_polygon.geom_type == 'Polygon':
        first_polygon_coords = first_polygon.exterior.coords
        # Display the coordinates
        for coord in first_polygon_coords:
            print(coord)
    else:
        print("The first geometry is not a Polygon.")
else:
    print("The combined GeoDataFrame is empty.")


# Calculating Coordinates for Each Pixel in Each Image

# Calculating Coordinates and Assigning Classes to Each Pixel

In this step, we will calculate the geographic coordinates for every pixel in each image within our dataset. This is crucial for spatial analysis as it allows us to accurately map the pixel data to real-world locations.

For each image, we will extract the georeferencing information (such as the affine transformation matrix) to determine the latitude and longitude of each pixel. These coordinates will then be stored in matrices, which can be used for further analysis and visualization.

After calculating the coordinates, we will perform a left join between the pixel matrices and the crop map polygons. This will allow us to assign each pixel with the corresponding class (e.g., crop type) it belongs to. This step is essential for classifying the pixels based on their geographic location within the mapped regions.

By mapping each pixel to its corresponding geographic coordinates and assigning the appropriate class, we can ensure that our analysis is both spatially accurate and contextually relevant, enabling precise alignment with other geospatial data layers and facilitating more detailed analysis.



In [ ]:
import numpy as np
from shapely.geometry import Point
import gc
import pandas as pd
import geopandas as gpd
import h5py
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

def calculate_pixel_coordinates(image_shape, central_coords, resolution_m):
    height, width = image_shape
    central_lat, central_lon = central_coords

    lat_res = resolution_m / 111320
    lon_res = resolution_m / (111320 * np.cos(np.radians(central_lat)))

    y_indices, x_indices = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')

    latitudes = central_lat + (y_indices - height // 2) * lat_res
    longitudes = central_lon + (x_indices - width // 2) * lon_res

    coords_matrix = np.stack([latitudes, longitudes], axis=-1)

    return coords_matrix

def create_point_matrix(coords_matrix):
    height, width = coords_matrix.shape[:2]
    point_matrix = np.empty((height, width), dtype=object)
    for i in range(height):
        for j in range(width):
            lat, lon = coords_matrix[i, j]
            point_matrix[i, j] = Point(lon, lat)
    return point_matrix

def process_single_image(image_shape, central_coords, resolution_m, idx, join_gdf):
    coords_matrix = calculate_pixel_coordinates(image_shape, central_coords, resolution_m)
    point_matrix = create_point_matrix(coords_matrix)
    height, width = point_matrix.shape[:2]

    points_list = [(i, j, point_matrix[i, j]) for i in range(height) for j in range(width)]
    points_df = pd.DataFrame(points_list, columns=['i', 'j', 'geometry'])
    gdf = gpd.GeoDataFrame(points_df, geometry='geometry')


    joined_gdf = gpd.sjoin(gdf, join_gdf, how='left', op='within')
    joined_gdf["crop_code"] = joined_gdf["crop_code"].fillna("un")

    result_matrix = np.empty((height, width), dtype=object)
    for row in joined_gdf.itertuples():
        result_matrix[row.i, row.j] = getattr(row, "crop_code", "un")

    return result_matrix

resolution_m = 10
batch_size = 20

drive_path = '/content/drive/My Drive/IntermediateResults'
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

hdf5_file_path = os.path.join(drive_path, 'all_data.h5')

with h5py.File(hdf5_file_path, 'w') as h5f:
    for idx, data in enumerate(ds):
        print(f"Processing image {idx + 1}/{len(ds)}")

        image_shape = data['thumbnail'].shape[1:3]
        central_coords = (data['meta']['centre_lat'], data['meta']['centre_lon'])

        result_matrix = process_single_image(image_shape, central_coords, resolution_m, idx, combined_gdf_transformed)

        group = h5f.create_group(f'image_{idx}')
        group.create_dataset('thumbnail', data=data['thumbnail'], compression='gzip')
        group.create_dataset('result_matrix', data=result_matrix, compression='gzip')

        meta_group = group.create_group('meta')
        for key, value in data['meta'].items():
            meta_group.attrs[key] = str(value)  # Convert all metadata values to strings

        print(f"Added result matrix to image {idx + 1}/{len(ds)}")

    print(f"Saved all data in HDF5 format at {hdf5_file_path}.")

# Clean up to free memory
vars_to_delete = ['result_matrix', 'coords_matrix', 'point_matrix', 'points_list', 'joined_gdf']
for var in vars_to_delete:
    if var in locals():
        del locals()[var]
gc.collect()
gc.collect()

print("All result matrices added to the dataset.")


# Creating Colored Masks and Organizing Data for Further Analysis

In this final step, we will generate colored masks for crop classification based on a predefined crop colors mapping. Each pixel in the image will be assigned a specific color corresponding to its crop class, resulting in a visual representation of crop types across the region.

### Color Mapping and Mask Creation
While we will use an initial color mapping for the crop classes, it's important to note that a more appropriate and thoughtful mapping should be developed. This will ensure that the colors used in the masks are intuitive and distinct, making the classification easier to interpret.

### Restricting to Major Crops
After creating the masks, we may also consider restricting the classification to only the major crops within the region. This step would simplify the classification task by focusing on the most significant crops, which could be more relevant for certain analyses or machine learning training.

### Saving Image and Mask Pairs
Finally, we will save each pair of the original image and its corresponding mask in a structured format. The data will be organized into folders by year and batch, ensuring that the dataset is well-organized and ready for further analysis or model training. This organization will facilitate easy access and management of the data as we proceed with our project.


In [ ]:
import h5py
import numpy as np
import pandas as pd
import os
from google.colab import drive
from PIL import Image

# Mount Google Drive
drive.mount('/content/drive')

def read_hdf5_in_batches(file_path, batch_size=10):
    with h5py.File(file_path, 'r') as h5f:
        num_images = len(h5f.keys())
        for start in range(0, num_images, batch_size):
            end = min(start + batch_size, num_images)
            data_batch = {}
            for i in range(start, end):
                group = h5f[f'image_{i}']
                result_matrix = group['result_matrix'][:]
                meta = {key: group['meta'].attrs[key] for key in group['meta'].attrs}

                # Read thumbnail and bands if they exist
                thumbnail = group['thumbnail'][:] if 'thumbnail' in group else None
                bands = group['bands'][:] if 'bands' in group else None

                data_batch[i] = {
                    'result_matrix': result_matrix,
                    'meta': meta,
                    'thumbnail': thumbnail,
                    'bands': bands
                }
            yield data_batch

def convert_to_dataframe(data_dict):
    # Create a list of dictionaries
    data_list = []
    for idx, data in data_dict.items():
        data_list.append({
            'result_matrix': data['result_matrix'],
            'meta': data['meta'],
            'thumbnail': data['thumbnail'],
            'bands': data['bands']
        })

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(data_list)

    return df

def save_image_pairs_to_drive(data_df, drive_path):
    if not os.path.exists(drive_path):
        os.makedirs(drive_path)

    crop_colors = {
        b'po': (255, 0, 0),    # Red
        b'or': (0, 255, 255),  # Cyan
        b'pe': (255, 0, 255),  # Magenta
        b'gr': (0, 255, 0),    # Green
        b'ot': (255, 255, 0),  # Yellow
        b'ma': (0, 0, 255),    # Blue
        b'sb': (128, 0, 128),  # Purple
        b'sw': (0, 128, 128),  # Teal
        b'wb': (255, 165, 0),  # Orange
        b'wo': (128, 128, 0),  # Olive
        b'ww': (75, 0, 130),   # Indigo
        b'sl': (255, 105, 180),# Hot Pink
        b'so': (64, 224, 208), # Turquoise
        b'fs': (0, 255, 127),  # Spring Green
        b'fw': (218, 112, 214),# Orchid
        b'un': (255, 255, 255) # White
    }

    for idx, row in data_df.iterrows():
        thumbnail = row['thumbnail']
        result_matrix = row['result_matrix']

        if thumbnail is not None and result_matrix is not None:
            # Print initial values of result_matrix
            print(f"Initial result_matrix values for image {idx}:\n{result_matrix}")

            # Print unique values in result_matrix to verify crop types
            unique_values = np.unique(result_matrix)
            print(f"Unique crop types in result_matrix for image {idx}: {unique_values}")

            # Create mask from result_matrix
            mask = np.zeros((*result_matrix.shape, 3), dtype=np.uint8)
            for crop_type, color in crop_colors.items():
                mask[result_matrix == crop_type] = color

            # Print the mask to ensure colors are assigned correctly
            unique_colors = np.unique(mask.reshape(-1, mask.shape[2]), axis=0)
            print(f"Unique colors in mask for image {idx}: {unique_colors}")

            # Convert arrays to images
            thumbnail_img = Image.fromarray((thumbnail * 255).astype(np.uint8).transpose(1, 2, 0))
            mask_img = Image.fromarray(mask)

            # Save images to Google Drive
            thumbnail_img.save(os.path.join(drive_path, f'thumbnail_{idx}.png'))
            mask_img.save(os.path.join(drive_path, f'mask_{idx}.png'))

            print(f"Saved pair {idx + 1}/{len(data_df)}")
        else:
            print(f"Skipping image {idx + 1}/{len(data_df)} due to missing data")

# Path to the saved HDF5 file
hdf5_file_path = '/content/drive/My Drive/IntermediateResults/all_data.h5'

# Path to save images on Google Drive
drive_path = '/content/drive/My Drive/MscProject/ImageMaskPairs2023'

# Read the HDF5 file in batches
batch_size = 10
batch_number = 0
for data_batch in read_hdf5_in_batches(hdf5_file_path, batch_size=batch_size):
    data_df = convert_to_dataframe(data_batch)
    save_image_pairs_to_drive(data_df, os.path.join(drive_path, f'batch_{batch_number}'))
    batch_number += 1
